# 🎨 MyVLM-Art 训练 (Clean Version)

⚠️ **要求**: GPU 运行时 (T4 或更高)

**运行顺序**:
1. 运行 Cell 1 (安装依赖)
2. 运行 Cell 2 (重启 - 会自动重启)
3. **重启后** 从 Cell 3 开始运行

## Cell 1: 安装依赖

⚠️ 运行完后必须重启 Runtime！

In [ ]:
%%capture
# 完全卸载旧版本
!pip uninstall -y bitsandbytes transformers accelerate -q

# 安装固定版本
!pip install transformers==4.36.2 --no-cache-dir -q
!pip install accelerate==0.25.0 --no-cache-dir -q
!pip install bitsandbytes==0.41.3 --no-cache-dir -q
!pip install peft>=0.2.0 -q
!pip install open-clip-torch -q
!pip install timm>=0.9.16 -q
!pip install sentence-transformers>=2.3.1 -q
!pip install omegaconf>=2.3.0 -q
!pip install pyrallis>=0.3.1 -q
!pip install loguru>=0.7.0 -q
!pip install wandb>=0.16.0 -q

In [ ]:
print("✅ 依赖安装完成")
print("⚠️ 现在运行下一个 Cell 重启 Runtime")

## Cell 2: 重启 Runtime

⚠️ **运行后自动重启，重启后跳过 Cell 1-2，直接从 Cell 3 开始**

In [ ]:
import os
os.kill(os.getpid(), 9)

---
# ⬇️⬇️⬇️ 重启后从这里开始 ⬇️⬇️⬇️
---

## Cell 3: 验证环境 + Clone 代码

In [ ]:
# 验证版本
import torch
import transformers
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 检查版本
assert transformers.__version__.startswith("4.36"), f"transformers 版本应为 4.36.x，当前: {transformers.__version__}"
print("\n✅ 环境验证通过")

In [ ]:
# Clone 代码
import os
import shutil

# TODO: 替换为你的仓库地址
REPO_URL = "https://github.com/YOUR_USERNAME/MyVLM_art.git"
BRANCH = "log-for-colab"

if os.path.exists("/content/MyVLM_art"):
    shutil.rmtree("/content/MyVLM_art")
    print("🗑️ 删除旧代码")

!git clone -b {BRANCH} {REPO_URL}
os.chdir("/content/MyVLM_art")
print(f"\n📁 当前目录: {os.getcwd()}")
!git log --oneline -3

## Cell 4: Wandb 登录 (可选)

In [ ]:
import wandb
wandb.login()
print("✅ Wandb 登录成功")

## Cell 5: 运行训练 🚀

In [ ]:
# ==================== 完整训练代码 ====================
import sys
import os
os.chdir("/content/MyVLM_art")
sys.path.insert(0, "/content/MyVLM_art")

# 导入并配置
from pathlib import Path
import torch
from myvlm.common import VLMType, PersonalizationTask, VLM_TO_LAYER
from configs.myvlm_art_config import MyVLMArtConfig
from vlms.llava_wrapper import LLaVAWrapper
from myvlm.myllava import MyLLaVA
from concept_graph.concept_embeddings.trainer import MultiTokenEmbeddingTrainer

# 配置参数
cfg = MyVLMArtConfig(
    concept_name="dataset",
    concept_identifier="test",
    vlm_type=VLMType.LLAVA,
    personalization_task=PersonalizationTask.CAPTIONING,
    output_root=Path("./outputs"),
    data_root=Path("./data"),
    optimization_steps=1,         # 快速测试，正式训练改为 100
    learning_rate=1.0,
    batch_size=1,                 # T4 建议 1-2
    reg_lambda=0.0,
    device='cuda',
    torch_dtype=torch.bfloat16,
    threshold=0.75,
    max_tokens_per_concept=1,
    max_concepts_per_sample=1,
    backoff_delta=0.05,
    val_subset_n=3,
    max_reason_tokens=64,
    # Wandb (设为 False 禁用)
    use_wandb=True,
    wandb_project="myvlm-art-test",
    wandb_run_name="quick-test",
)

# 训练
print("🔄 加载 LLaVA 模型...")
vlm = LLaVAWrapper(device=cfg.device, torch_dtype=cfg.torch_dtype)

print("🔧 初始化 MyLLaVA...")
myvlm = MyLLaVA(vlm, layer=VLM_TO_LAYER[cfg.vlm_type], concept_name=cfg.concept_name, cfg=cfg)

print("🚀 开始训练...")
trainer = MultiTokenEmbeddingTrainer(cfg=cfg, myvlm=myvlm, dataset_builder=None)
checkpoints = trainer.train()

# 保存
from datetime import datetime
from zoneinfo import ZoneInfo
out_dir = cfg.output_root / cfg.concept_name / f"seed_{cfg.seed}"
out_dir.mkdir(parents=True, exist_ok=True)
ts = datetime.now(ZoneInfo('America/New_York')).strftime("%Y%m%d_%H%M%S")
save_path = out_dir / f"checkpoints_{cfg.concept_name}_seed_{cfg.seed}_{ts}.pt"
torch.save(checkpoints, save_path)

print(f"\n✅ 训练完成!")
print(f"💾 检查点: {save_path}")

## Cell 6: 保存到 Google Drive (可选)

In [ ]:
from google.colab import drive
import shutil
from pathlib import Path

drive.mount('/content/drive')

save_dir = Path("/content/drive/MyDrive/MyVLM_art_outputs")
save_dir.mkdir(parents=True, exist_ok=True)

shutil.copytree("/content/MyVLM_art/outputs", save_dir / "outputs", dirs_exist_ok=True)
print(f"✅ 已保存到: {save_dir / 'outputs'}")